In [1]:
import pandas as pd
import numpy as np
import sqlite3
import json
from pathlib import Path

## Define tags you want to filter by

In [44]:
tag_filters = {
    'task': 'classification'
}

## Set path to pf database

In [45]:
# Set path to promptflow database
con = sqlite3.connect('/home/maxca/.promptflow/pf.sqlite')
con.row_factory = sqlite3.Row


## Filter runs and extract metrics

In [46]:
cur = con.cursor()
stmt = '''
SELECT * FROM run_info WHERE json_extract (tags, '$.task') = :task
'''
res = cur.execute(stmt, tag_filters).fetchall()

metric_list = []
for r in res:
    flow_path = Path(json.loads(r['properties'])['output_path'])
    mpath = flow_path / 'metrics.json'
    if mpath.exists():
        with open(flow_path / 'metrics.json') as fp:
            metrics = json.load(fp)
            metric_list.append(metrics)
    else:
        metrics = {}
    metrics['display_name'] = r['display_name']
    metrics['start_time'] = r['start_time']
    metrics['description'] = r['description']
    for k,v in json.loads(r['tags']).items():
        metrics[k] = v
    #metrics['tags'] = json.loads(r['tags'])
            
df = pd.DataFrame.from_dict(metric_list).dropna(subset='f1_score')
df


,f1_score,precision_score,recall_score,display_name,start_time,description,task,labels,dataset
0,1.0,1.0,1.0,classify_binary_variant_0_20250826_122620_824384,2025-08-26T12:26:21.817715+02:00,None,classification,binary,NaN
1,1.0,1.0,1.0,classify_binary_variant_0_20250826_122641_228398,2025-08-26T12:26:42.219660+02:00,None,classification,binary,X
2,1.0,1.0,1.0,classify_binary_variant_0_20250826_122813_542772,2025-08-26T12:28:14.567724+02:00,Binary classification of relevance to climate ...,classification,binary,X
3,1.0,1.0,1.0,classify_binary_variant_0_20250826_124347_857233,2025-08-26T12:43:49.409873+02:00,Binary classification of relevance to climate ...,classification,binary,X
